In [ ]:
pip install scikit-image

In [ ]:
# import data processing and visualisation libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# import image processing libraries
import cv2
import skimage
from skimage.transform import resize

# import tensorflow and keras
import tensorflow as tf
from tensorflow import keras
import os

print("Packages imported...")

In [ ]:
# Librerias para construir el modelo
import os
import numpy as np
import cv2
import skimage.transform
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Flatten, Dense, MaxPooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 32
imageSize = 64
target_dims = (imageSize, imageSize, 3)
num_classes = 29

train_len = 87000
train_dir = '/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/'

def get_data(folder):
    X = np.empty((train_len, imageSize, imageSize, 3), dtype=np.float32)
    y = np.empty((train_len,), dtype=np.int)
    cnt = 0
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['A']:
                label = 0
            elif folderName in ['B']:
                label = 1
            elif folderName in ['C']:
                label = 2
            elif folderName in ['D']:
                label = 3
            elif folderName in ['E']:
                label = 4
            elif folderName in ['F']:
                label = 5
            elif folderName in ['G']:
                label = 6
            elif folderName in ['H']:
                label = 7
            elif folderName in ['I']:
                label = 8
            elif folderName in ['J']:
                label = 9
            elif folderName in ['K']:
                label = 10
            elif folderName in ['L']:
                label = 11
            elif folderName in ['M']:
                label = 12
            elif folderName in ['N']:
                label = 13
            elif folderName in ['O']:
                label = 14
            elif folderName in ['P']:
                label = 15
            elif folderName in ['Q']:
                label = 16
            elif folderName in ['R']:
                label = 17
            elif folderName in ['S']:
                label = 18
            elif folderName in ['T']:
                label = 19
            elif folderName in ['U']:
                label = 20
            elif folderName in ['V']:
                label = 21
            elif folderName in ['W']:
                label = 22
            elif folderName in ['X']:
                label = 23
            elif folderName in ['Y']:
                label = 24
            elif folderName in ['Z']:
                label = 25        
            else:
                label = 26
            for image_filename in os.listdir(folder + folderName):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file).reshape((-1, imageSize, imageSize, 3))
                    
                    X[cnt] = img_arr
                    y[cnt] = label
                    cnt += 1
    return X,y
X_train, y_train = get_data(train_dir)
print("Images successfully imported...")

In [ ]:
print("The shape of X_train is : ", X_train.shape)
print("The shape of y_train is : ", y_train.shape)

In [ ]:
print("The shape of one image is : ", X_train[0].shape)

In [ ]:
plt.imshow(X_train[300])
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

In [ ]:
# One-Hot - codificar los datos categóricos
from tensorflow.keras.utils import to_categorical
y_cat_train = to_categorical(y_train, num_classes)
y_cat_test = to_categorical(y_test, num_classes)

In [ ]:
# Checking the dimensions de todas las variables
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_cat_train.shape)
print(y_cat_test.shape)

In [ ]:
import gc
del X_data
del y_data
gc.collect()

In [ ]:
def residual_block(x, filters, kernel_size=5, stride=1):
    shortcut = x

    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    if shortcut.shape[-1] != filters or stride != 1:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

input_shape = (imageSize, imageSize, 3)
input_layer = Input(shape=input_shape)
x = Conv2D(128, kernel_size=5, strides=2, padding='same')(input_layer)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
x = residual_block(x, filters=64, stride=2)
x = Flatten()(x)
x = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=x)
model.summary()

In [ ]:
# Compila el modelo con la misma configuración que tenía durante el entrenamiento original
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Crear generador de imágenes con volteo vertical en Y
datagen = ImageDataGenerator(vertical_flip=True)

# Ajustar el generador de imágenes a los datos de entrenamiento
datagen.fit(X_train)

# Entrenar el modelo utilizando el generador de imágenes
history = model.fit(datagen.flow(X_train, y_cat_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) / batch_size,
                    epochs=12,
                    validation_data=(X_test, y_cat_test))

## Evaluacion

In [ ]:
metrics = pd.DataFrame(model.history.history)
print("The model metrics are")
metrics

In [ ]:
metrics[['loss','val_loss']].plot()
plt.show()

In [ ]:
metrics[['accuracy','val_accuracy']].plot()
plt.show()

In [ ]:
predictions = model.predict_classes(X_test)
print("Predictions done...")

In [ ]:
from keras.models import load_model
model.save('ASL.h5')
print("Modelo guardado exitosamente")

### Codigo para compilar el escript con el H5 del modelo

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('ASL.h5')

labels = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'Nada'
]

def extract_features(image):
    image = cv2.resize(image, (64, 64))  
    image = np.expand_dims(image, axis=0)  
    return image / 255.0  

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.rectangle(frame, (50, 50), (350, 350), (0, 255, 0), 2) 
    crop_frame = frame[50:350, 50:350]

    features = extract_features(crop_frame)
    pred = model.predict(features)
    prediction_label = labels[np.argmax(pred)]

    cv2.rectangle(frame, (0, 0), (640, 40), (0, 255, 0), -1) 
    accu = "{:.2f}".format(np.max(pred) * 100)
    cv2.putText(frame, f'{prediction_label}  {accu}%', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("Sign Language Recognitipon", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
